In [2]:
!pip install awscli --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!aws configure set aws_access_key_id ""
!aws configure set aws_secret_access_key ""
!aws configure set default.region us-east-2

In [4]:
import mlflow

mlflow.set_tracking_uri("http://ec2-3-133-149-243.us-east-2.compute.amazonaws.com:5000/")

In [5]:
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

2026/02/05 12:10:56 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sajjad-mlflow-bucket-26/3', creation_time=1770275457044, experiment_id='3', last_update_time=1770275457044, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [6]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [7]:
df = pd.read_csv("reddit_preprocessing.csv")

df.shape

(36793, 2)

In [12]:
df[df['clean_comment'].isna()]['category'].value_counts()

Series([], Name: count, dtype: int64)

In [11]:
df.dropna(inplace=True)

In [14]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    
    x_train, x_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

    x_train = vectorizer.fit_transform(x_train)
    x_test = vectorizer.transform(x_test)

    with mlflow.start_run() as run:
        
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

ngram_ranges = [(1, 1), (1, 2), (1, 3)]
max_features = 5000

for ngram_range in ngram_ranges:
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2026/02/05 13:39:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/02/05 13:39:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2026/02/05 13:39:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-3-133-149-243.us-east-2.compute.amazonaws.com:5000/#/experiments/3/runs/0d63701160fc4234aad446fa1007a987.
2026/02/05 13:39:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-133-149-243.us-east-2.compute.amazonaws.com:5000/#/experiments/3.
2026/02/05 13:40:30 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/02/05 13:40:30 WARNING mlflow.models.model: Model logg